# 1 An Introduction JuMP

Let's consider a trivial LP to introduce the basics of JuMP.

\begin{matrix}
\min & z = x_1 + x_2 & \\
\text{s.t.} & & \\
    & x_1 + x_2 \le 1 \\
\text{and} & x_1, x_2 \geq 0.
\end{matrix}

In [1]:
using JuMP, GLPK

Defining our model.
 
*Model* is a type defined by JuMP. The objective function, all variables and constraints are associated with a unique model object.

In [2]:
m = Model(with_optimizer(GLPK.Optimizer))
@variable(m, x_1 >= 0)
@variable(m, x_2 >= 0)
@objective(m, Min, x_1 + x_2)
@constraint(m, x_1 + x_2 <= 1)
;

x_1 + x_2 ≤ 1.0

We can print our model to see what it looks like.

In [3]:
println("The LP to be solved is:")
m

The LP to be solved is:


A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x_1, x_2

Now we solve the model.

In [5]:
optimize!(m)
println("Solution;")
println("Objective value: ", objective_value(m))
println("x_1 = ", value(x_1))
println("x_2 = ", value(x_2))

Solution;
Objective value: 0.0
x_1 = 0.0
x_2 = 0.0


# 2 General Problems
## 2.1 A production and distribution problem

Firm ABC has two production plants located in different parts of the country which must produce and distribute a product to three regional warehouses.
Plant one has unit production costs of R4,5 and production capacity of 2000 units. Plant two has unit production costs of R6 and a production capacity if 3000 units.
The three warehouses have demands of 500, 2000 and 900 units of the product respectively.

The transportation costs per unit are as follows;

 
 From Plant | To Warehouse 1 | To Warehouse 2 | To Warehouse 3
 ---------- | -------------- | -------------- | --------------
 A          | 1,3            | 1,9            | 1,8
 B          | 1,7            | 1,2            | 1,4

The problem is how to meet all demand while minimising total production and distribution costs.


### Modelling the problem
We model the problem by first defining the data that is given, next we build the model with the associated objective, variables and constraints.

In [6]:
# Defining the model data.
origin = ["Plant A", "Plant B"]
destination = ["Warehouse 1", "Warehouse 2", "Warehouse 3"]

supply = [2000 3000]
demand = [500 2000 900]

total_cost = [
    5.8 6.4 6.3;
    7.7 7.2 7.4
]

# Building the model.
ModelABC = Model(with_optimizer(GLPK.Optimizer))

@variable(ModelABC, send[1:length(origin), 1:length(destination)] >= 0)

@objective(ModelABC, Min, sum(total_cost[i, j] * send[i, j] for i in 1:length(origin), j in 1:length(destination)))

@constraint(ModelABC, [i in 1:length(origin)], sum(send[i, j] for j in 1:length(destination)) <= supply[i])
@constraint(ModelABC, [j in 1:length(destination)], sum(send[i, j] for i in 1:length(origin)) == demand[j])

# Display model for sanity check!
println("THE LP TO BE SOVLVED IS:")
ModelABC

THE LP TO BE SOVLVED IS:


A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: send

### Solving the model
The code below the status call prints the results in an intelligible form.

In [16]:
optimize!(ModelABC)

if termination_status(ModelABC) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(ModelABC))")
    for i in 1:length(origin)
        println("From $(origin[i]):")
        for j in 1:length(destination)
            println(" send $(value(send[i, j])) to $(destination[j])")
        end
    end
else
    println("No solution")
end

Optimal objective value: 22490.0
From Plant A:
 send 500.0 to Warehouse 1
 send 600.0 to Warehouse 2
 send 900.0 to Warehouse 3
From Plant B:
 send 0.0 to Warehouse 1
 send 1400.0 to Warehouse 2
 send 0.0 to Warehouse 3


## 2.2 A Diet "Problem"

The following problem is taken from Operations Research - Applications And Algorithms 4th edition Winston, Wayne L which I'll simply refer to as _Winston_ going forward.

Except from chapter 3 of _Winston_;
My diet requires that all the food I eat come from one of the four “basic food groups” (chocolate cake, ice cream, soda, and cheesecake). At present, the following four foods are available for consumption: brownies, chocolate ice cream, cola, and pineapple cheese- cake. Each brownie costs 50¢, each scoop of chocolate ice cream costs 20¢, each bottle of cola costs 30¢, and each piece of pineapple cheesecake costs 80¢. Each day, I must in- gest at least 500 calories, 6 oz of chocolate, 10 oz of sugar, and 8 oz of fat. The nutri- tional content per unit of each food is shown in Table 2. Formulate a linear programming model that can be used to satisfy my daily nutritional requirements at minimum cost.

Here is a reproduction of Table 2:

Type of Food                  |Calories |Chocolate (Ounces) |Sugar (Ounces) |Fat (Ounces) 
----------                    |---------|--------------     |-------------- |------------
Brownie                       |400      |3                  |2              |2
Chocolate ice cream (1 scoop) |200      |2                  |2              |4
Cola (1 bottle)               |150      |0                  |4              |1
Pineapple cheesecake (1 piece)|500      |0                  |4              |5

The problem is to formulate a linear programming model that satisfies the daily "nutritional" requirements at minimum cost.

In [17]:
# Defining the data
foods = ["brownie", "ice cream", "soda", "cheesecake"]
nutriCategories = ["calories", "chocolate", "sugar", "fat"]
numFoods = length(foods)
numCategories = length(nutriCategories)

cost = [50, 20, 30, 80]
nutritionValues = [
    400 3 2 2;
    200 2 2 4;
    150 0 4 1;
    500 0 4 5
]

#A = [
#    400 200 150 500;
#    3 2 0 0;
#    2 2 4 4;
#    2 4 1 5
#]
minNutrition = [500, 6, 10, 8]
;

4-element Array{Int64,1}:
 500
   6
  10
   8

In [19]:
# Building the model
DietModel = Model(with_optimizer(GLPK.Optimizer))

# Decision variables
@variable(DietModel, x[1:numFoods] >= 0)

# Objective function
@objective(DietModel, Min, cost'*x) # dot(cost, x) not working, so used '*

# Model constraints
for j in 1:numCategories
    @constraint(DietModel, nutritionValues[:,j]'*x >= minNutrition[j]) # dot(nutritionValues[:,j], x) not working so used '*
end

# Display model for sanity check!
println("THE LP TO BE SOVLVED IS:")
DietModel

THE LP TO BE SOVLVED IS:


A JuMP Model
Minimization problem with:
Variables: 4
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

Solving the model:

In [20]:
function PrintSolution(status, foods, x)
    println("RESULTS:")
    if termination_status(ModelABC) == MOI.OPTIMAL
        println(" Min Cost = $(objective_value(DietModel))")
        for i in 1:length(foods)
            println("  $(foods[i]) = $(value(x[i]))")
        end
    else
        println("  No solution")
    end
    println("")
end

status = optimize!(DietModel)
PrintSolution(status, foods, x)

RESULTS:
 Min Cost = 90.0
  brownie = 0.0
  ice cream = 3.0
  soda = 1.0000000000000002
  cheesecake = 0.0



We use the following function to get the constraints final values by calculating $A^T\mathbf x$, where $A$ = nutritionValues:

In [29]:
function PrintDietNutrition()
    finalNutrition = transpose(nutritionValues)*value.(x[:]) # JuMP.value.(x[:])
    println("The optimal diet provides:")
    for i in 1:length(nutriCategories)
        println(" $(nutriCategories[i]) = $(finalNutrition[i])")
    end
end

PrintDietNutrition()

The optimal diet provides:
 calories = 750.0
 chocolate = 6.0
 sugar = 10.0
 fat = 13.0
